<a href="https://colab.research.google.com/github/Delta357/API_GPT-4/blob/main/chatgpt_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install the necessary libraries
!pip install openai pandas numpy sklearn awscli boto3 sagemaker

In [ ]:
openai.api_key = ''

In [ ]:
# Set the model and prompt
model_engine = "text-davinci-003"
prompt = "Criar um modelo machine learning de nlp com para analise de sentimento com pyspark mostar matriz de confussão e acuracia do modelo mostrar um exemplo em python"

# Set the maximum number of tokens to generate in the response
max_tokens = 2000

# Generate a response
completion = openai.Completion.create(
    engine=model_engine,
    prompt=prompt,
    max_tokens=max_tokens,
    temperature=0.5,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
)

# Print the response
print(completion.choices[0].text)

In [ ]:
!pip install pyspark

In [ ]:
!pip install spark

In [ ]:
#importando bibliotecas
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
import pyspark
from pyspark.sql import SparkSession

# Create SparkSession
spark = SparkSession.builder.master("local[1]").appName("SparkByExamples.com").getOrCreate()
sc=spark.sparkContext

In [ ]:
#dados de treino
train = spark.createDataFrame([
    (0, "I am very happy today", 1.0),
    (1, "I am very sad today", 0.0),
    (2, "I am very angry today", 0.0),
    (3, "I am very excited today", 1.0),
    (4, "I am very disappointed today", 0.0),
    (5, "Respiratory syncytial virus, or RSV, is a common respiratory virus that is the leading cause of bronchiolitis", 1.0),
    (6, "As pioneers in asthma, we continue to push the boundaries of science. We’re joining leading experts at ATS2021", 0.0),
    (7, "It takes innovation to transform healthcare experiences for patients, but innovation doesn’t happen in isolation", 0.0),
    (8, "Every day billions of people are trying to balance living their lives with the limitations posed by chronic disease", 0.0),
    (9, "An estimated 37 million people live with chronic kidneydisease in the US and up to 90% are undiagnosed.",1.0),
    (10,"RSV is the leading cause of bronchiolitis and pneumonia cases in infants under one year of age and remains",0.0)

], ["id", "text", "label"])

In [ ]:
#tokenizando
tokenizer = Tokenizer(inputCol="text", outputCol="words")
wordsData = tokenizer.transform(train)

In [ ]:
#criando o modelo
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures")
featurizedData = hashingTF.transform(wordsData)

In [ ]:
#gerando o modelo
idf = IDF(inputCol="rawFeatures", outputCol="features")

idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)

In [ ]:
#criando o label
label_stringIdx = StringIndexer(inputCol = "label", outputCol = "indexed")
label_model = label_stringIdx.fit(rescaledData)
final_data = label_model.transform(rescaledData)

In [ ]:
#dividindo os dados
(train_data, test_data) = final_data.randomSplit([0.75, 0.25])

In [ ]:
#criando o modelo de regressão logistica
lr = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0)

#criando o pipeline
pipeline = Pipeline(stages=[lr])

#treinando o modelo
model = pipeline.fit(train_data)

In [ ]:
#testando o modelo
predictions = model.transform(test_data)
predictions

In [ ]:
#calculando a acuracia
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
accuracy = evaluator.evaluate(predictions)

#resultado
print("Acurácia do modelo:", accuracy)

In [ ]:
# matriz de confusão
predictions.groupBy("label","prediction").count().show()